In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import math
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

df = pd.read_csv('/kaggle/input/playersnew/IPL-fantasy-stats - Fantasy-stats (1).csv')

In [ ]:
df['Matches'] = df['Matches'].replace(0, 1)
df['AveragePoints'] = df['Points'] / df['Matches']
df.head()

In [ ]:
df.describe()

In [ ]:
def verify_team_structure(pt):
    neg_score = 0
    # WicketKeeper 1-4
    if pt['WicketKeeper'] < 1:
        neg_score -= 1 - pt['WicketKeeper']
    if pt['WicketKeeper'] > 4:
        neg_score -= pt['WicketKeeper'] - 4
    
    # Batman 3-6
    if pt['Batsman'] < 3:
        neg_score -= 3- pt['Batsman']
    if pt['Batsman'] > 6:
        neg_score -= pt['Batsman'] - 6
    
    # AllRounder 1-4
    if pt['AllRounder'] < 1:
        neg_score -= 1 - pt['AllRounder']
    if pt['AllRounder'] > 4:
        neg_score -= pt['AllRounder'] - 4
    
    # Bowler 3-6
    if pt['Bowler'] < 3:
        neg_score -= 3 - pt['Bowler']
    if pt['Bowler'] > 6:
        neg_score -= pt['Bowler'] - 6
        
    return neg_score
    

In [ ]:
def get_scores(players):
    foreigners = 0
    team_count = {1: 0, 2: 0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0}
    average_score_sum = 0
    player_type = {'Batsman': 0, 'WicketKeeper': 0, 'AllRounder': 0, 'Bowler': 0}
    cost = 0
    
    for idx in players:
        player = df.iloc[idx]
        player_type[player['PlayerType']] += 1
        foreigners += player['Foreigner']
        team_count[player['TeamId']] += 1
        average_score_sum += player['AveragePoints']
        cost += player['Cost']
        
    neg_score = verify_team_structure(player_type)
    if cost > 100:
        neg_score -= cost - 100
    if foreigners > 4:
        neg_score -= foreigners - 4
#     print('Team_count', team_count)
    for value in team_count.values():
        if value > 7:
            neg_score -= value - 7
    
    
#     print(average_score_sum, neg_score)
#     if neg_score < 0:
#         return neg_score
#     else:
#         return average_score_sum
#     return average_score_sum + (neg_score*10)
    return average_score_sum, neg_score

In [ ]:
def getFitness(players):
    average_score_sum, neg_score = get_scores(players)
#     if neg_score < 0:
#         return neg_score
#     else:
#         return average_score_sum
    return average_score_sum + (neg_score*10)

In [ ]:
def crossoverA(team1,team2):
    combined_team  = list(set(team1+team2))
    new_team = random.sample(combined_team, 11)
    return new_team

def crossoverB(team1,team2):
    team1.sort()
    team2.sort()
    
    selected = []
    for i in range(11):
        if team1[i] in selected:
            selected.append(team2[i])
        elif team2[i] in selected:
            selected.append(team1[i])
        else:
            players = [team1[i],team2[i]]
            selected.append(random.choice(players))
            
    return(selected)
    

In [ ]:
def crossover_population(population, parents):
    children = []
    for idx in range(0, len(parents), 2):
        parentA = population[parents[idx]]
        parentB = population[parents[idx+1]]
        
        child = crossoverA(parentA, parentB)
        children.append(child)
        
    return population + children

In [ ]:
def mutateA(team):
    members_to_change = random.randint(1, 3)
    idxs = random.sample(range(11), members_to_change)
    
    for idx in idxs:
        while True:
            new_member = random.randint(0, 196)
            if new_member not in team:
                team[idx] = new_member
                break
    
    return team
    
def mutateB(team):
    return team

In [ ]:
def mutate_population(population, mutate_ratio):
    population_count = len(scores)
    team_ids = random.sample(range(1, population_count), math.floor(population_count*mutate_ratio))
    
    for idx in team_ids:
        population[idx] = mutateA(population[idx])
    
    return population

In [ ]:
def generate_poplation(population_count):
    population = []
    for i in range(population_count):
        team = random.sample(range(196), 11)
        population.append(team)
        
    return population

In [ ]:
def get_population_score(population):
    scores = []
    for team in population:
        fitness = getFitness(team)
        scores.append(fitness)
        
    #sort population based on scores
    population = [x for _, x in sorted(zip(scores, population), reverse=True)]
    scores.sort(reverse=True)
        
    return population, scores

In [ ]:
def parent_selection(scores, crossover_ratio):
    population_count = len(scores)
    parents = random.choices(range(population_count), scores, k = math.floor(population_count*crossover_ratio))
    
    return parents

In [ ]:
def print_team(team):
    for i in team:
        print(i)
        print(df.iloc[i])

In [ ]:
team1 = [1, 7, 5, 35, 24, 6, 10, 44, 40, 32, 29]
team2 = [1, 43, 6, 13, 34, 5, 11, 41, 23, 3, 19]
print(getFitness(team1),getFitness(team2))
team = crossoverB(team1,team2)
print(team)
print(getFitness(team))

In [ ]:
GEN_COUNT = 100
POPULATION_COUNT = 2000
CROSSOVER_RATIO = 0.4
MUTATE_RATIO = 0.1
NEW_POPULATION_RATIO = 0.1

best_teams = []
best_team_scores = []

best_team = []
best_team_score = 0

population = generate_poplation(POPULATION_COUNT)
population, scores = get_population_score(population)
# print(population, scores)

for gen in range(GEN_COUNT):
    print('Gen: ', gen, 'Best score: ', scores[0])
    
    # save best team
    # best_teams.append(population[0])
    # best_team_scores.append(scores[0])
    if scores[0] > best_team_score:
        pos, neg = get_scores(population[0])
        if neg == 0:
            print(scores[0], population[0], max(scores))
            best_team_score = scores[0]
            best_team = population[0]
    
    # choose parents
    parent_ids = parent_selection(scores, CROSSOVER_RATIO)
    
    # crossover
    population = crossover_population(population, parent_ids)
    
    # Add extra individuals
    population += generate_poplation(math.floor(POPULATION_COUNT*NEW_POPULATION_RATIO))
    
    # remove extra individuals
    population, scores = get_population_score(population)
    population = population[:POPULATION_COUNT]
    scores = scores[:POPULATION_COUNT]
    
    # mutate
    population = mutate_population(population, MUTATE_RATIO)
    population, scores = get_population_score(population)
    
    
    
    
    

# save best team
best_teams.append(population[0])
best_team_scores.append(scores[0])

In [ ]:
get_scores([172, 176, 162, 109, 140, 135, 167, 117, 40, 175, 129]), best_team_score

In [ ]:
print_team([172, 176, 162, 109, 140, 135, 167, 117, 40, 175, 129])